In [1]:
import numpy as np 
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

In [2]:
tf.__version__

'2.3.0'

In [3]:
def build_model(num_classes=265, img_width=512, img_height=256):
    inputs = layers.Input(shape=(img_height, img_width, 3))
    model = EfficientNetB4(include_top=False, input_tensor=inputs)

    x = layers.GlobalMaxPooling2D(name="max_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.5
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)

    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    model.compile(
        optimizer=Adam(learning_rate=1e-3), 
        loss="categorical_crossentropy", 
        metrics=[tfa.metrics.F1Score(num_classes = num_classes, average = 'micro')]
    )

    return model

In [4]:
model = build_model()
model.load_weights( './data/efn_b4_ckpt_tf_2_3_0.h5')

In [5]:
model.save('./data/efn_b4_tf_2_3_0.h5')